## This script is used to tkae HEC-RAS output from a 2d model, where an error is thrown for TW cells exceeding the elevation of a SA/2D storage area's weir elevation.

#### We take the output, and convert it to a table that can be used in Excel.

From Excel, The elevations are increased from the TW Cell Elevation by .1', those are then used to replace the elevations being used in the Weir Table of the SA/2D Connection.

In excel, Add the update +.1' station/elevations back into the weir table, sort by station, then add an extra column to ensure only the max elevation is used for each station using a formula like: `=IF(sta=sta2,MAX(elev:elev2),elev)` to ensure the row does not have a smaller elevation value than the next row. Then, remove duplicates.

In [18]:
weir_data = """
	   138.87	  253.390	   11493	  253.844	   12734     12736

	   163.17	  253.160	   11494	  253.781	   12736     12738

	   759.77	  253.000	   11499	  253.344	   12749     12558

	   820.71	  253.120	   11192	  253.750	   12373     12372

	   843.56	  253.460	   11192	  253.750	   12373     12372

	   936.37	  253.030	   11193	  253.344	   38775     12375

	   971.15	  253.050	   11346	  253.243	   12375     12561

	  1130.10	  253.170	   11838	  253.469	   13149     13151

	  1399.00	  252.990	   12586	  253.324	   13998     14000

	  1435.09	  252.970	   12587	  253.190	   14000     14002

	  1489.81	  253.000	   12589	  253.215	   14004     14006

	  1555.47	  253.030	   12399	  253.250	   13791     13794

	  1803.52	  252.990	   12787	  253.131	   14230     14232

	  1862.15	  253.380	   12789	  253.691	   14234     14236

	  2434.97	  253.310	   14188	  253.576	   15586     15815

	  2481.82	  253.130	   14386	  253.250	   15815     16043

	  2650.88	  253.100	   14582	  253.238	   16266     16044

	  2843.47	  253.030	   13587	  253.500	   15130     38796

	  2992.60	  253.000	   13391	  253.625	   14906     14908

	  3031.52	  253.100	   13392	  253.750	   14908     14910

	  3033.07	  253.486	   13392	  253.750	   14908     14910

	  3289.51	  253.160	   12595	  253.531	   14015     14017

	  3321.00	  253.120	   12596	  253.594	   14017     14019

	  3563.46	  253.000	   12215	  253.192	   13590     13387

	  3616.26	  253.110	   11843	  253.227	   13160     38786

	  3885.48	  252.970	   11850	  253.170	   13174     13176

	  4037.54	  253.070	   11199	  253.183	   12386     38778

	  4096.19	  253.030	   11199	  253.183	   38778     12169

	  4126.12	  253.240	   11198	  253.428	   12168     38777

	  4287.39	  253.000	   11043	  253.153	   12149     12144

	  4312.56	  253.340	   11042	  253.593	   12144     12143

	  4428.39	  253.340	   11505	  253.482	   38783     12759

	  4582.74	  252.970	   34298	  253.594	   38803     13589

	  4605.15	  253.000	   12214	  253.251	   13589     13381

	  4632.57	  252.970	   12214	  253.251	   13589     13381

	  4649.82	  253.160	   12028	  253.565	   13381     13380

	  4723.77	  253.090	   12026	  253.352	   38787     13155

	  4759.05	  253.000	   12026	  253.352	   38787     13155

	  4789.91	  252.990	   11840	  253.095	   13155     12955

	  5035.77	  253.000	   34316	  253.330	   12132     38709

	  5291.72	  252.970	   10488	  253.110	   11456     11289

	  5581.26	  253.330	    9547	  253.531	   10352     38773

	  5606.36	  253.030	    9547	  253.531	   38773     10200

	  5858.79	  253.560	    8735	  253.870	    9429     38772

	  6039.47	  252.970	    8327	  253.719	    8981      8835

	  6116.42	  252.970	    7922	  253.149	    8551     38769

	  6217.90	  253.060	    7923	  253.691	    8554      8412

	  6292.26	  252.970	    7661	  253.125	    8272      8135

	  6662.93	  252.970	    6082	  253.076	    6586      6451

	  6760.89	  253.940	    6081	  255.625	    6446      6585

	  6763.75	  255.449	    6081	  255.625	    6446      6585

	  6772.03	  254.650	    6199	  255.717	    6585      6715

	  6784.30	  254.940	    6199	  255.717	    6585      6715

	  6841.46	  253.180	    6318	  254.638	    6715      6842

	  6851.20	  253.680	    6318	  254.638	    6715      6842

	  6865.39	  253.550	    6437	  254.458	    6842      6970

	  6877.20	  253.900	    6437	  254.458	    6842      6970

	  6895.85	  253.180	    6437	  254.458	    6842      6970

	  7057.00	  253.060	    6555	  254.009	    7097      7096

	  7069.80	  253.580	    6555	  254.009	    7097      7096

	  7123.59	  253.665	    6554	  253.784	    7096     38767

	  7130.68	  253.000	    6554	  253.784	   38767      6965

	  7149.93	  253.190	    6554	  253.784	   38767      6965

	  7172.89	  252.970	    6554	  253.784	   38767      6965

	  7345.51	  252.970	    5962	  253.185	    6440      6328

	  7588.90	  253.000	    5366	  253.327	    5756      5637

	  7639.81	  253.000	    5268	  253.359	    5637      5636

	  7720.21	  254.420	    4942	  255.156	    5276      5275

	  7724.74	  255.053	    4942	  255.156	    5276      5275

	  7730.60	  254.790	    4941	  255.633	    5275      5274

	  7737.26	  254.340	    4941	  255.633	    5275      5274

	  7747.60	  254.710	    4941	  255.633	    5275      5274

	  7774.98	  254.250	    4940	  255.673	    5274      5273

	  7786.06	  254.720	    4940	  255.673	    5274      5273

	  7794.93	  254.310	    4939	  255.802	    5273      5272

	  7796.98	  255.489	    4939	  255.802	    5273      5272

	  7801.59	  254.560	    5061	  255.644	    5272      5404

	  7812.68	  253.810	    5061	  255.644	    5272      5404

	  7827.91	  254.470	    5061	  255.644	    5272      5404

	  7849.64	  253.560	    5061	  255.644	    5272      5404

	  7872.97	  254.620	    5060	  255.728	    5404      5403

	  7890.19	  255.380	    5059	  256.323	    5403      5402

	  7909.55	  255.840	    5059	  256.323	    5403      5402

	  7920.31	  256.470	    5058	  256.984	    5402      5401

	  7933.21	  256.060	    5171	  256.653	    5401      5527

	  7959.22	  255.720	    5171	  256.653	    5401      5527

	  7969.15	  254.840	    5171	  256.653	    5401      5527

	  7995.24	  255.000	    5170	  256.469	    5527      5526

	  8003.68	  254.320	    5170	  256.469	    5527      5526

	  8016.35	  254.120	    5169	  256.656	    5526      5525

	  8069.13	  254.790	    5167	  256.467	    5524      5523

	  8086.02	  255.090	    5167	  256.467	    5524      5523

	  8105.01	  254.770	    5166	  255.992	    5523      5522

	  8121.14	  254.970	    5166	  255.992	    5523      5522

	  8151.45	  254.640	    5165	  255.750	    5522      5521

	  8206.33	  254.600	    5163	  255.220	    5520      5519

	  8229.54	  254.430	    5162	  255.125	    5519      5518

	  8252.76	  253.700	    5161	  255.021	    5518      5517

	  8276.43	  254.120	    5161	  255.021	    5518      5517

	  8478.91	  254.530	    4918	  254.896	    5250      5116

	  8611.94	  254.310	    4561	  254.487	    4814      4700

	  8717.82	  253.220	    4357	  254.049	    4572      4508

	  8958.36	  253.690	    3949	  254.729	    4149      4094

	  8972.44	  253.480	    3949	  254.729	    4149      4094

	  8983.93	  253.470	    3949	  254.729	    4149      4094

	  8993.57	  254.150	    3949	  254.729	    4149      4094

	  9010.00	  253.490	    3896	  254.867	    4094      4039

	  9021.73	  253.280	    3896	  254.867	    4094      4039

	  9038.16	  253.080	    3844	  254.863	    4039      3987

	  9045.20	  253.520	    3844	  254.863	    4039      3987

	  9056.38	  253.260	    3844	  254.863	    4039      3987

	  9075.82	  253.780	    3794	  255.605	    3987      3935

	  9085.53	  254.920	    3794	  255.605	    3987      3935

	  9095.56	  254.690	    3744	  255.719	    3935      3884

	  9130.53	  255.970	    3695	  256.281	    3884      3835

	  9584.78	  261.950	    3250	  262.222	    3412      3411

	  9599.67	  262.120	    3249	  262.491	    3411      3410

	  9635.84	  262.840	    3248	  263.114	    3410      3409

	  9695.41	  262.660	    3247	  262.969	    3409     38764
"""

In [19]:
from io import StringIO
import pandas as pd


# Clean up and load
df = pd.read_csv(StringIO(weir_data),
                sep=r"\s+",
                header=0,
                names=["Weir_Sta", "Weir_Elv", "Cell", "Cell_Elv", "Face_Pt1", "Face_Pt2"])

print(df)



     Weir_Sta  Weir_Elv   Cell  Cell_Elv  Face_Pt1  Face_Pt2
0      163.17    253.16  11494   253.781     12736     12738
1      759.77    253.00  11499   253.344     12749     12558
2      820.71    253.12  11192   253.750     12373     12372
3      843.56    253.46  11192   253.750     12373     12372
4      936.37    253.03  11193   253.344     38775     12375
..        ...       ...    ...       ...       ...       ...
113   9130.53    255.97   3695   256.281      3884      3835
114   9584.78    261.95   3250   262.222      3412      3411
115   9599.67    262.12   3249   262.491      3411      3410
116   9635.84    262.84   3248   263.114      3410      3409
117   9695.41    262.66   3247   262.969      3409     38764

[118 rows x 6 columns]
